## Explore tome and train models

Let's read in that tome and train a model to go from number of footsteps to rank, obviously.

In [ ]:
from pureskillgg_makenew_pyskill.notebook import setup_notebook

In [ ]:
setup_notebook()

In [ ]:
# %load ../usual_suspects.py
# pylint: disable=unused-import
import time
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pureskillgg_dsdk.tome import TomeCuratorFs

pd.set_option("display.max_columns", 150)
pd.set_option("display.max_rows", 150)
pd.set_option("display.min_rows", 150)
# pd.set_option('display.float_format', '{:.4f}'.format)

curator = TomeCuratorFs()


In [ ]:
df = curator.get_dataframe('footsteps_by_rank')
keyset = curator.get_keyset('footsteps_by_rank')

In [ ]:
print(len(df),len(keyset))

In [ ]:
df.head(25)

## Training a model left as exercise to user

:) 